<a href="https://colab.research.google.com/github/MHoseinHoushmand/Clustering_by_SLFA/blob/main/Clustering_by_SLFA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pdb
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from numpy.linalg import norm
import operator
import random

In [ ]:
categories = [  #Select 4 categories from fetch_20newsgroups dataset
    "alt.atheism",
    "comp.graphics",
    "sci.space",
    "rec.sport.hockey",
]

dataset = fetch_20newsgroups( #Preprocessing before using dataset
    remove=("headers", "footers", "quotes"),
    subset="all",
    categories=categories,
    shuffle=True,
    random_state=42,
)

labels = dataset.target[0:500]
unique_labels, category_sizes = np.unique(labels, return_counts=True)
true_k = unique_labels.shape[0]
print(f"{len(dataset.data)} documents - {true_k} categories")
print(labels)

3758 documents - 4 categories
[3 2 0 0 1 1 2 3 2 1 3 1 3 0 0 2 1 0 3 1 0 2 1 0 2 2 1 1 3 3 0 1 1 2 3 3 2
 1 1 1 2 1 2 3 2 0 0 2 2 1 3 0 1 2 2 3 1 0 1 2 0 1 3 1 3 2 0 3 1 1 0 3 3 0
 1 0 2 0 2 1 2 2 0 2 3 2 1 2 0 2 0 1 1 3 3 2 0 2 0 3 1 0 0 1 2 3 3 0 3 3 2
 1 0 0 1 3 2 2 1 2 2 0 2 3 2 1 0 1 2 3 2 2 2 3 3 0 3 3 0 0 2 0 1 3 1 2 1 1
 3 2 3 3 0 1 3 3 2 2 2 3 2 0 2 0 1 3 3 1 0 2 2 2 2 3 2 3 1 1 1 1 2 3 3 1 2
 1 1 2 3 1 0 2 2 2 0 2 3 3 2 1 1 0 3 2 0 3 2 3 0 2 3 3 0 1 3 1 0 3 0 0 2 3
 3 3 1 1 1 3 1 0 3 1 0 3 2 3 3 0 0 1 1 3 2 2 2 1 3 0 2 3 3 0 1 3 2 0 3 2 0
 0 0 1 2 3 3 3 3 2 1 3 2 2 2 1 0 3 2 2 3 0 3 2 1 0 2 1 2 2 2 0 2 2 2 1 2 3
 0 0 3 2 3 0 3 3 0 0 0 0 0 0 2 0 2 3 3 3 3 0 3 0 3 2 1 3 1 2 2 0 3 0 1 1 1
 1 2 2 3 1 1 2 0 0 3 1 0 1 1 2 1 3 3 3 3 2 2 3 1 3 1 0 0 0 2 3 2 2 1 0 1 1
 2 0 3 3 3 3 1 0 0 0 3 2 2 1 2 2 0 1 3 3 3 1 2 0 2 1 1 1 2 3 3 3 0 1 1 0 3
 2 1 3 3 0 3 2 2 3 3 2 2 1 1 2 2 3 2 0 3 0 3 2 3 1 3 3 1 1 0 2 1 1 3 0 2 0
 3 3 3 2 1 2 3 1 2 3 3 1 1 1 3 3 1 3 2 2 0 1 2 2 1 0 0 3 1 1 1 1 1 3 1

In [ ]:
# Vectorize all document as their term frequency(tfidf score)
def docs_as_tfidf(docs):
  vectorizer = TfidfVectorizer(
     max_df=0.5, #Removing terms that are used in more than 50% of articles
     min_df=5,   #Removing terms that are not used in less than 10 of articles
     stop_words="english",
     #  max_features=1000,
  )
  docs_vector = vectorizer.fit_transform(docs)
  return docs_vector.toarray()

In [ ]:
def cosin_sim(a,b):
   return cosine_similarity([a], [b])[0][0]

In [ ]:
class Cluster:
     def __init__(self):
         documents = []
         size = len(self.documents)
         doc_mean = []

     def get_doc_mean(self):
        self.doc_mean = np.average(self.documents, axis=0)
        return self.doc_mean

     def SSE(self):     #Sum of squared error(SSE) as similarity of each documents with the cluster mean in document
         doc_mean = self.doc_mean
         sse=0
         for doc in self.documents:
             sse += cosin_sim(doc,doc_mean)**2
             sse = sse/self.size
         return sse


In [ ]:
class Frog:
     def __init__(self,answer,docs_vector,n_clusters):
        answer = answer
        value = self.fitness(docs_vector,n_clusters)
        tuplefrom = (answer,self.value)


     def build_clusters(self,docs_vector,n_clusters):
          clusters = []
          for i in range(n_clusters):
             cluster = Cluster()
             clusters.append(cluster)
          for j in range(len(self.answer)):
             if -1 < self.answer[j]:
                clusters[self.answer[j]].documents.append(docs_vector[j])
          return clusters


     def WC(self,clusters):   #Calculate similarity within clusters
        WC = 0
        for cluster in clusters:
            WC += cluster.SSE()
        return WC


     def BC(self,doc_means):#Calculate similarity between clusters
          BC=0
          size = len(doc_means)
          for i in range(size):
              for j in range(i+1,size):
                  BC += cosin_sim(doc_means[i],doc_means[j])**2
          return BC


     def fitness(self,docs_vector,n_clusters):
           doc_means = []
           clusters = self.build_clusters(docs_vector,n_clusters)
           for i in range(n_clusters):
               doc_means.append(
                  np.average(clusters[i], axis=0)
               )
           wc = self.WC(clusters) #Calculate similarity within clusters
           bc = self.BC(doc_means) #Calculate similarity between clusters
           fitness = wc/bc
           return fitness


     def cross_over(self,frog_b): #perform 2 points cross over
           frog_size = len(self.answer)
           points = sorted(np.random.choice(np.arange(0,frog_size), size=2, replace=False))
           answer1 = self.answer[:points[0]] + frog_b.answer[points[0]:points[1]] + self.answer[points[1]:]
           answer2 = frog_b.answer[:points[0]] + self.answer[points[0]:points[1]] + frog_b.answer[points[1]:]
           child1 = Frog(answer1)
           child2 = Frog(answer2)
           if child1.value > child2.value: # return best child
               return (child1 , child1.value)
           else :
               return (child2 , child2.value)

     def mutation(self,clusters_size):
         new_ans = list(self.answer)
         size = int(len(self.answer)/4)
         indexes = np.random.choice(np.arange(0,len(self.answer)), size=size, replace=False)
         values= [random.randint(0, 3) for _ in range(size)]
         for i in range(size):
            new_ans[indexes[i]] = values[i]
         child = Frog(tuple(new_ans))
         return  child


In [ ]:
class Memplex:
     def __init__(self):
         frogs = []

     def best(self):
        if len(self.frogs) > 0
           return self.frogs[0]

     def worst(self,frogs):
        if len(self.frogs) > 0
           return self.frogs[len(self.frogs)-1]

     def frogs_to_remove(self,sub_memplex):
         for item in sub_memplex:
            self.frogs.remove(item)
         return self

     def add_frogs(self,submemplex):
         self.frogs = self.frogs + submemplex.frogs






In [ ]:
class Submemplex:
     def __init__(self, memplex ,n_submemplex ):
         frogs = []
         prob_list = []
         keys = []
         for i in range(memplex_size):
             for j in range(2*(memplex_size-i)):
                 prob_list.append(i)
         k=0
         while(k!=n_submemplex):
            index = random.choice(prob_list)
            key = memplex.frogs[index][0]
            if key not in keys:
                frogs.append(memplex.frogs[index])
                keys.append(key)
                k+=1

     def best(self):
        if len(self.frogs) > 0
           return self.frogs[0]

     def worst(self,frogs):
        if len(self.frogs) > 0
           return self.frogs[len(self.frogs)-1]

     def add_frog(self,frog):
        index = 0
        for tfrog in self.frogs:
            if frog.value >  tfrog.value:
                self.frogs.insert(index,frog)
            index += 1

In [ ]:
class Population:
     def __init__(self,n_clusters, n_population ,n_docs, docs_vector):
        answers = np.random.randint(0, n_clusters, size=(n_population , n_docs))
        frogs = []
        for answer in range(answers):
           frog = Frog(answer,docs_vector,n_clusters)
           frogs.append(frog)

     def clear_frogs(self):
        self.frogs.clear()

     # Best frog in population
     def global_best(self,memplexes):
        local_bests = []
        for memplex in memplexes:
            local_best =  max(memplex, key=lambda frog: frog.value)
            local_bests.append(local_best)
        global_best =  max(local_bests, key=lambda frog: frog.value)
        return global_best

     def shufeling(self,memplexes):
         output = []
         for memplex in memplexes:
            output = output+ memplex
         self.frogs = output


In [ ]:
class SLFA:
     def  __init__(self,n_clusters, max_iteration, docs):
        n_clusters = n_clusters
        max_iteration =  max_iteration
        docs = docs
        n_population = 400 # Frogs number
        memplex_size = 20 #define as m
        n_memplex = 20 #define as n
        local_iteration = 10 #Iteration As local search
        n_submemplex = 5
        docs_vector = docs_as_tfidf(docs)  #Convert documents as tfidf values


     def create_memplexes(self,population,n_memplex):
             population = sorted(population, key=lambda frog: frog.value)
             memplexes = []
             for i in range(memplex_num):
                 memplex = Memplex()
                 memplexes.append(memplex)
             for i in range(self.n_population):
                 memplexes[i % memplex_num].frogs.append(population[i])
             return memplexes



     def search(self):
        population = Population(self.n_clusters, self.n_population, self.n_docs, self.docs_vector)
        memplexes = self.Create_memplexes(population,self.n_memplex)
        for i in range(max_iteration):
           memplexes = self.create_memplexes(population,self.n_memplex)
           population.clear_frogs()
           for j in range(self.n_memplex):
               submemplex = Submemplex(memplexes[j],self.n_submemplex)
               memplexes[j].frogs_to_remove(submemplex)
               for k in range(local_iteration):
                    lfrog_best = submemplex.best()
                    lfrog_worst = submemplex.worst()
                    frog_out = lfrog_worst.cross_over(lfrog_best)
                    if (lfrog_worst.value < frog_out.value ):
                         submemplex.frogs.remove(lfrog_worst)
                         submemplex.add_frog(frog_out)
                    else:
                         gfrog_best = population.global_best(memplexes)
                         frog_out = cross_over(gfrog_best[0],lfrog_worst[0])
                         if (lfrog_worst[1]<frog_out[1]):
                              submemplex.remove(lfrog_worst)
                              submemplex.add_frog(frog_out)
                         else:
                              frog_out = gfrog_best.mutation(n_clusters)
                              submemplex.remove(lfrog_worst)
                              submemplex.append(frog_out)
               memplexes[j].add_frogs(submemplex)
           gfrog_best = population.global_best(memplexes)
           population.shufeling(memplexes)


In [ ]:
docs = dataset.data[0:500]
slfa = SLFA(4,50,docs)


In [ ]:
population_size = 400 # Frogs number
memplex_num = 20 #define as m
memplex_size = 20 #define as n
max_iteration = 100 #Total Iteration
local_iteration = 10 #Iteration As local search
n_clusters = 4
docs = dataset.data[0:500]
docs_vector = docs_as_tfidf(docs)
print(list(docs_vector[0]))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19326683304032288, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [ ]:
#Calculate sum of squared error(SSE) as similarity of each documents with the cluster mean in document
def SSE(cluster,doc_mean):
  size = len(cluster)
  sse=0
  for doc in cluster:
    sse += cosin_sim(doc,doc_mean)**2
  sse = sse/size
  return sse

In [ ]:
#Calculate similarity between clusters
def BC(doc_means):
   BC=0
   size = len(doc_means)
   for i in range(size):
      for j in range(i+1,size):
          BC += cosin_sim(doc_means[i],doc_means[j])**2
   return BC

In [ ]:
#Calculate similarity within clusters
def WC(clusters):
    WC = 0
    for cluster in clusters:
        doc_mean = np.average(cluster, axis=0)
        WC += SSE(cluster,doc_mean)
    return WC

In [ ]:
def build_clusters(answer,docs_vector,n_clusters):
   clusters = []
   for i in range(n_clusters):
       clusters.append([])
   for j in range(len(answer)):
       if -1 < answer[j]:
        clusters[answer[j]].append(docs_vector[j])
   return clusters

In [ ]:
def fitness(answer,docs_vector,size):
   doc_means = []
   clusters = build_clusters(answer,docs_vector,size)
   for i in range(size):
       doc_means.append(
          np.average(clusters[i], axis=0)
       )
   wc = WC(clusters) #Calculate similarity within clusters
   bc = BC(doc_means) #Calculate similarity between clusters
   fitness = wc/bc
   return fitness

In [ ]:
#perform 2 points cross over
def cross_over(answer_a,answer_b):
    frog_size = len(answer_a)
    points = sorted(np.random.choice(np.arange(0,frog_size), size=2, replace=False))
    child1 = answer_a[:points[0]] + answer_b[points[0]:points[1]] + answer_a[points[1]:]
    child2 = answer_b[:points[0]] + answer_a[points[0]:points[1]] + answer_b[points[1]:]
    fitness1 =  fitness(child1 ,docs_vector,n_clusters)
    fitness2 = fitness(child2 ,docs_vector,n_clusters)
    if fitness1 > fitness2: # return best child
       return (child1 , fitness1)
    else :
       return (child2 , fitness2)

In [ ]:
def best_and_worst(answers):
     best =  max(answers, key=lambda x:x[1])
     worst = min(answers, key=lambda x:x[1])
     return best , worst

In [ ]:
# Best frog in population
def global_best(memplexes):
     local_bests = []
     for memplex in memplexes:
         local_best =   max(memplex, key=lambda x:x[1])
         local_bests.append(local_best)
     global_best = max(local_bests, key=lambda x:x[1])
     return global_best

In [ ]:
def frogs_to_remove(sub_memplex , memplex):
    for item in sub_memplex:
       memplex.remove(item)
    return memplex

In [ ]:
def mutation(global_best,clusters_size):
    new_ans = list(global_best)
    size = int(len(global_best)/4)
    indexes = np.random.choice(np.arange(0,len(global_best)), size=size, replace=False)
    values= [random.randint(0, 3) for _ in range(size)]
    for i in range(size):
      new_ans[indexes[i]] = values[i]
    return tuple(new_ans)

In [ ]:
def Create_memplexes(population,memplex_num):
     population = list(reversed(sorted(population, key=lambda x: x[1])))
     memplexes = []
     population_size = len(population)
     for i in range(memplex_num):
         memplexes.append([])
     for i in range(population_size):
         memplexes[i % memplex_num].append(population[i])
     return memplexes

In [ ]:
def create_submemplex(memplex,memplex_size, submemplex_size):
    sub_memplex = []
    prob_list = []
    keys = []
    for i in range(memplex_size):
       for j in range(2*(memplex_size-i)):
          prob_list.append(i)
    k=0
    while(k!=submemplex_size):
       index = random.choice(prob_list)
       key = memplex[index][0]
       if key not in keys:
           sub_memplex.append(memplex[index])
           keys.append(key)
           k+=1
    return sub_memplex


In [ ]:
#sh
def shufeling(memplexes):
    output = []
    for memplex in memplexes:
        output = output+ memplex
    return output

In [ ]:
def frog_leaping_search(docs_vector):
                     answers=np.random.randint(0, n_clusters, size=(population_size , len(docs)))
                     population = []
                     i=0
                     for answer in answers:
                        i+=1
                        frog = (tuple(answer),fitness(answer,docs_vector,n_clusters))
                        population.append(frog)
                        print(i,frog[1])
                     for i in range(max_iteration):
                        memplexes = Create_memplexes(population, memplex_num)
                        population.clear()
                        for j in range(memplex_num):
                            print(i,j,len(memplexes[j]))
                            sub_memplex = create_submemplex(memplexes[j],memplex_size, 5)
                            memplexes[j] =  frogs_to_remove(sub_memplex,memplexes[j])
                            for k in range(local_iteration):
                                ans_best, ans_worst = best_and_worst(sub_memplex)
                                ans_out = cross_over(ans_best[0],ans_worst[0])
                                if (ans_worst[1]<ans_out[1]):
                                    sub_memplex.remove(ans_worst)
                                    sub_memplex.append(ans_out)
                                else:
                                    g_best = global_best(memplexes)
                                    ans_out = cross_over(g_best[0],ans_worst[0])
                                    if (ans_worst[1]<ans_out[1]):
                                       sub_memplex.remove(ans_worst)
                                       sub_memplex.append(ans_out)
                                    else:
                                        ans_out = mutation(g_best[0],n_clusters)
                                        fitness_out = fitness(ans_out,docs_vector,n_clusters)
                                        sub_memplex.remove(ans_worst)
                                        sub_memplex.append((ans_out , fitness_out))
                            memplexes[j]= memplexes[j]+sub_memplex
                            print(i,j,len(memplexes[j]))
                        g_best = global_best(memplexes)
                        print(g_best[0])
                        print(g_best[1])
                        population = shufeling(memplexes)
                        print(len(population))
                     return g_best, g_value, population

In [ ]:

g_best, g_value,population = frog_leaping_search(docs_vector)
true = 0
size = len(dataset.data)
print(labels)
print(list(g_best))

1 0.04153801244198741
2 0.038705126975120796
3 0.0393280032847259
4 0.04044760737533567
5 0.04162716949285829
6 0.04137034125582893
7 0.03966440660320541
8 0.04086095183883073
9 0.040232877486579235
10 0.03840422792476109
11 0.04026613942081062
12 0.040893627223279175
13 0.03896096054731363
14 0.04076740024544587
15 0.038350056943709035
16 0.039100375557676106
17 0.03985727809625182
18 0.040169041860323565
19 0.0398524139980088
20 0.03974948305853104
21 0.03987611727915797
22 0.042356223745631125
23 0.03983827695558289
24 0.039956119723999775
25 0.040706079460927094
26 0.03940071778029402
27 0.038399590714279876
28 0.04057936487489854
29 0.03947215394108695
30 0.041718535459844386
31 0.039916965357627746
32 0.0398572423132548
33 0.040183818216792615
34 0.039657688998500695
35 0.039176128952048445
36 0.04032133192073559
37 0.04012883160435416
38 0.0396563606734277
39 0.04105232920630757
40 0.039609391605790634
41 0.039567670909102765
42 0.04040460611524923
43 0.03974270873774837
44 0.04

KeyboardInterrupt: ignored